<a href="https://colab.research.google.com/github/DataNata/Recommendation-system/blob/main/Recommend_system_Courses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Рекомендательная система по продаже курсов

In [4]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras


Файл содержит данные по продажам курсов - в файле ужу отфильрованы покупатели, купившие более одного курса

In [5]:
course_count=pd.read_csv('/content/purchase.csv')
course_count=course_count[['user_id','purchase_date','resource_id']]
course_count.head(5)

,user_id,purchase_date,resource_id
0,51,2017-01-06 21:31:53.507,516
1,51,2018-06-22 17:20:49.080,1099
2,6117,2017-06-30 17:36:47.875,356
3,6117,2017-06-30 17:36:47.875,357
4,6117,2018-08-01 05:01:45.031,1125


In [6]:
list_of_courses=course_count['resource_id'].unique()

In [7]:
list_of_courses

array([ 516, 1099,  356,  357, 1125,  553, 1147,  361, 1138, 1140,  551,
        745,  568,  514,  517,  566,  562,  563,  363,  511,  509, 1144,
        672,  552,  571,  513, 1141,  862,  744,  679,  750,  800,  569,
        840,  765, 1187, 1103, 1100,  564,  502,  865,  764, 1139, 1186,
        366,  367,  519,  809,  515,  912,  489,  523,  864, 1146, 1101,
        776,  753,  671,  829,  490, 1102,  803,  659,  909,  794,  518,
        907,  777,  908,  360,  813,  835,  741,  752,  814, 1115, 1161,
       1116,  863,  743,  572,  504,  810, 1124, 1128,  742, 1104,  503,
        664,  507,  570, 1185, 1198,  365,  359,  791, 1156,  362, 1184,
        911,  358, 1160,  757,  508, 1181,  755, 1145, 1188,  756,  866,
        749,  368,  364,  834, 1152,  670, 1199,  836, 1201, 1129, 1182,
        902,  837, 1200,  833,  830])

In [8]:
#Найдем количество различных пар курсов встречающихся вместе в покупках клиентов
users_purchase_list = course_count.groupby(['user_id'])['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()

In [11]:
users_purchase_list

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"
...,...,...
12651,2179430,"[566, 750]"
12652,2186581,"[794, 864, 1129]"
12653,2187601,"[356, 553, 571, 765, 912]"
12654,2188926,"[515, 743]"


In [12]:
#Найдем количество клиентов купивших более одного курса:
len(users_purchase_list)

12656

In [13]:
#Подготовим датафрейм result_list для подсчета уникальных пар купленных курсов: т.е. подготовим Кортедж пар ID курсов
import itertools
result_list = list()
# проход  по спискам resource_id
for courses in users_purchase_list['resource_id']:
    # проход циклом по содержимому каждого списка с делением на пары
    for course_pair in itertools.combinations(courses, 2):
        #результат запишем в список result_list
        result_list.append(course_pair)


In [14]:
result_set = set(result_list)

In [15]:
result_set

{(np.int64(504), np.int64(814)),
 (np.int64(564), np.int64(865)),
 (np.int64(489), np.int64(1100)),
 (np.int64(791), np.int64(907)),
 (np.int64(863), np.int64(1161)),
 (np.int64(356), np.int64(570)),
 (np.int64(776), np.int64(864)),
 (np.int64(814), np.int64(1156)),
 (np.int64(515), np.int64(743)),
 (np.int64(909), np.int64(1128)),
 (np.int64(835), np.int64(1099)),
 (np.int64(514), np.int64(864)),
 (np.int64(659), np.int64(1102)),
 (np.int64(356), np.int64(835)),
 (np.int64(504), np.int64(670)),
 (np.int64(568), np.int64(1100)),
 (np.int64(777), np.int64(865)),
 (np.int64(514), np.int64(1129)),
 (np.int64(1103), np.int64(1115)),
 (np.int64(569), np.int64(570)),
 (np.int64(356), np.int64(1100)),
 (np.int64(757), np.int64(1115)),
 (np.int64(519), np.int64(569)),
 (np.int64(516), np.int64(909)),
 (np.int64(829), np.int64(1140)),
 (np.int64(502), np.int64(745)),
 (np.int64(515), np.int64(864)),
 (np.int64(502), np.int64(516)),
 (np.int64(513), np.int64(791)),
 (np.int64(514), np.int64(756)

In [16]:
#Найдем сколько различных пар курсов встречаются вместе в покупках клиентов :
len(result_set)

3989

Найдем самую популярную пару: воспользуемся библиотекой collections, в которой есть счетчик Counter() , в результате на первом месте у вас получится самая популярная пара:

In [17]:
import collections
c = collections.Counter()
for pairs in result_list:
    c[pairs]+=1
print(c)

Counter({(np.int64(551), np.int64(566)): 797, (np.int64(515), np.int64(551)): 417, (np.int64(489), np.int64(551)): 311, (np.int64(523), np.int64(551)): 304, (np.int64(566), np.int64(794)): 290, (np.int64(489), np.int64(515)): 286, (np.int64(490), np.int64(566)): 253, (np.int64(490), np.int64(551)): 247, (np.int64(570), np.int64(752)): 247, (np.int64(569), np.int64(572)): 216, (np.int64(515), np.int64(523)): 213, (np.int64(553), np.int64(745)): 212, (np.int64(489), np.int64(523)): 206, (np.int64(569), np.int64(840)): 204, (np.int64(514), np.int64(551)): 200, (np.int64(516), np.int64(745)): 199, (np.int64(515), np.int64(566)): 195, (np.int64(489), np.int64(566)): 188, (np.int64(504), np.int64(572)): 184, (np.int64(572), np.int64(840)): 178, (np.int64(551), np.int64(552)): 177, (np.int64(507), np.int64(570)): 172, (np.int64(490), np.int64(809)): 163, (np.int64(489), np.int64(490)): 152, (np.int64(507), np.int64(752)): 150, (np.int64(523), np.int64(552)): 144, (np.int64(490), np.int64(515)

In [18]:
# выведем самую популярную пару:
popula_par = c.most_common(1)[0][0]
popula_par


(np.int64(551), np.int64(566))

Построим таблицу с рекомендациями:Столбец 1. Курс, к которому идёт рекомендация Столбец 2. Курс для рекомендации № 1 (самый популярный) Столбец 3. Курс для рекомендации № 2 (второй по популярности).
Создадим словарь пар курсов:


In [19]:
dict_c = dict(c)

In [20]:
dict_c

{(np.int64(516), np.int64(1099)): 25,
 (np.int64(356), np.int64(357)): 100,
 (np.int64(356), np.int64(1125)): 44,
 (np.int64(357), np.int64(1125)): 52,
 (np.int64(553), np.int64(1147)): 16,
 (np.int64(361), np.int64(1138)): 40,
 (np.int64(1125), np.int64(1140)): 1,
 (np.int64(551), np.int64(745)): 138,
 (np.int64(553), np.int64(745)): 212,
 (np.int64(551), np.int64(1138)): 14,
 (np.int64(553), np.int64(568)): 83,
 (np.int64(514), np.int64(517)): 10,
 (np.int64(514), np.int64(566)): 138,
 (np.int64(517), np.int64(566)): 21,
 (np.int64(363), np.int64(511)): 99,
 (np.int64(363), np.int64(562)): 77,
 (np.int64(363), np.int64(563)): 33,
 (np.int64(511), np.int64(562)): 55,
 (np.int64(511), np.int64(563)): 19,
 (np.int64(562), np.int64(563)): 53,
 (np.int64(568), np.int64(745)): 102,
 (np.int64(509), np.int64(553)): 48,
 (np.int64(509), np.int64(745)): 59,
 (np.int64(1125), np.int64(1144)): 22,
 (np.int64(509), np.int64(568)): 46,
 (np.int64(509), np.int64(672)): 5,
 (np.int64(568), np.int64

In [21]:
sorted_pairs = {k: v for k, v in sorted(dict_c.items(), key=lambda item: item[0], reverse=False)}
sorted_pairs

{(np.int64(356), np.int64(357)): 100,
 (np.int64(356), np.int64(360)): 1,
 (np.int64(356), np.int64(361)): 17,
 (np.int64(356), np.int64(366)): 15,
 (np.int64(356), np.int64(367)): 12,
 (np.int64(356), np.int64(368)): 1,
 (np.int64(356), np.int64(489)): 26,
 (np.int64(356), np.int64(490)): 13,
 (np.int64(356), np.int64(502)): 17,
 (np.int64(356), np.int64(503)): 1,
 (np.int64(356), np.int64(508)): 1,
 (np.int64(356), np.int64(509)): 5,
 (np.int64(356), np.int64(513)): 1,
 (np.int64(356), np.int64(514)): 35,
 (np.int64(356), np.int64(515)): 21,
 (np.int64(356), np.int64(516)): 16,
 (np.int64(356), np.int64(517)): 3,
 (np.int64(356), np.int64(519)): 14,
 (np.int64(356), np.int64(523)): 24,
 (np.int64(356), np.int64(551)): 48,
 (np.int64(356), np.int64(552)): 7,
 (np.int64(356), np.int64(553)): 5,
 (np.int64(356), np.int64(564)): 3,
 (np.int64(356), np.int64(566)): 21,
 (np.int64(356), np.int64(568)): 6,
 (np.int64(356), np.int64(569)): 2,
 (np.int64(356), np.int64(570)): 1,
 (np.int64(35

In [22]:
#всего уникальных пар :
len(sorted_pairs)


3989

Оформим рекомендательную систему функцией recommend:

In [24]:
def recommend(course):
  course_list = [] #создаем пустой списо
  for i in sorted_pairs.keys():
    if i[0] == course: # если первый элемент пары курсов = id курсу, который мы подаем на вход функции
            course_list.append((i, sorted_pairs[i])) #добавляем в список кортеж, который состоит из пары курсов и количества
  sorted_course_list = sorted(course_list, key=lambda x: x[1], reverse=True)#сортируем список в порядк убывания по количеству
  return sorted_course_list[:2]#выводим только 2 значения, в которых содержаться курсы для рекомендации №1 и №2


In [25]:
recommend(356)

[((np.int64(356), np.int64(571)), 103), ((np.int64(356), np.int64(357)), 100)]

In [26]:
recomendations_df = pd.DataFrame(columns =['Рекомендация №1', 'Рекомендация №2'])
for i in list_of_courses :
    if len(recommend(i)) == 2: # когда функция выдает 2 рекомендации
        recomendations_df.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]]
    elif len(recommend(i)) == 1: # когда функция выдает 1 рекомендацию
        recomendations_df.loc[i] = [recommend(i)[0][0][1], np.nan]
    else:
        recomendations_df.loc[i] = [np.nan, np.nan] # когда функция не выдает рекомендаций


In [27]:
recomendations_df.head(5)

,Рекомендация №1,Рекомендация №2
516,745.0,553.0
1099,1139.0,1187.0
356,571.0,357.0
357,571.0,1125.0
1125,1186.0,1144.0


In [ ]:
#данные можем сохранить в excel файл
recomendations_df.to_excel(r'recommendation_table.xlsx')